In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time
import mmap
import random
import pickle
import os
import argparse

In [2]:
import gpt_model_kit as GPT

In [3]:
# Hiperparameters
batch_size = 64
n_embd = 384
n_head = 1
n_layers = 1
dropout = 0.2
max_iters = 200
eval_iters = 50
block_size = 128    #Context Window Lenght (how many words/tokes the model can receive and respond)


model_filename = 'jdm_model.pkl'

#Dataset Path
valid_file_path = "D:/AI_Datasets/LLM/openwebtext/val_split.txt"
train_file_path = "D:/AI_Datasets/LLM/openwebtext/train_split.txt"
vocab_file_path = 'D:/AI_Datasets/LLM/openwebtext/vocab.txt'


In [4]:
#Checking if CUDA is available
#The available device will be configured to be used by the model during the training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [5]:
# Criando modelo
model = GPT.GptModel(n_embd, n_head, n_layers, block_size, dropout, vocab_file_path, device)
m = model.to(device)

In [6]:
model.train_model( 
    max_iters = max_iters, 
    eval_iters = eval_iters, 
    batch_size = batch_size, 
    train_file_path = train_file_path, 
    valid_file_path = valid_file_path, 
    vocab_file_path = vocab_file_path
)

Params:
 Max Iterations: 200
 Eval Iterations: 50
 batch Size: 64
 CUDA Acceleration: 12.1

Starting Training...
 step: 0, train loss 10.472, val loss 10.462
 step: 50, train loss 2.743, val loss 2.688
 step: 100, train loss 2.503, val loss 2.492
 step: 150, train loss 2.420, val loss 2.348
 Final Loss: 2.4874987602233887


In [7]:
model.save_model(model_filename)

Model saved
